<h1 align="center" style="color:#96d9f0;">Computational Intelligence for
Optimization - Project</h1>
<h3 align="center" style="color:#96d9f0;">Group AW - Wedding Seating Optimization</h3>

---

### <span style="color:#96d9f0;">Group Members</span>

<table>
  <thead style="color:#96d9f0;">
    <tr>
      <th>Name</th>
      <th>Email</th>
      <th>Student ID</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Afonso Dias</td>
      <td>20211540@novaims.unl.pt</td>
      <td>20211540</td>
    </tr>
    <tr>
      <td>Isabel Duarte</td>
      <td>20240545@novaims.unl.pt</td>
      <td>20240545</td>
    </tr>
    <tr>
      <td>Pedro Campino</td>
      <td>20240537@novaims.unl.pt</td>
      <td>20240537</td>
    </tr>
    <tr>
      <td>Rita Matos</td>
      <td>20211642@novaims.unl.pt</td>
      <td>20211642</td>
    </tr>
  </tbody>
</table>

In [1]:
import sys
sys.path.append('..')

In [2]:
import random
from copy import deepcopy
from library.solution import Solution
import csv
import os
import pandas as pd
import numpy as np
from itertools import combinations

In [3]:
#future improvements: test random fitnesses to set a baseline average to compare our results

In [4]:
#No swap mutation within the same table

In [3]:
# Import csv
import csv
import os
import pandas as pd

relations_matrix = np.loadtxt('seating_data.csv', delimiter=',', skiprows=1)
relations_matrix = relations_matrix[ : , 1:]

#relationship_matrix = pd.read_csv('seating_data.csv')
#relationship_matrix.drop(columns='idx', inplace=True)

relations_matrix 

array([[   0., 5000.,    0., ...,    0.,    0.,    0.],
       [5000.,    0.,  700., ...,    0.,    0.,    0.],
       [   0.,  700.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,  700.,  700.],
       [   0.,    0.,    0., ...,  700.,    0.,  900.],
       [   0.,    0.,    0., ...,  700.,  900.,    0.]])

In [34]:
class WSSolution(Solution):
    def __init__(self, repr=None, relations_matrix=relations_matrix, nr_of_tables=8):
        self.relations_matrix = relations_matrix
        self.nr_of_tables = nr_of_tables
        self.table_capacity = len(relations_matrix) // nr_of_tables
        super().__init__(repr=repr)

    def random_initial_representation(self):
        tables = []
        left_idxs = [idx for idx in range(len(self.relations_matrix))]
        for i in range(self.nr_of_tables):
            tables.append([])
            for j in range(self.table_capacity):
                idx = random.choice(left_idxs)
                left_idxs.remove(idx)
                # Check if idx is already in a table
                while any(idx in table for table in tables):
                    idx = random.randint(0, len(self.relations_matrix) - 1)
                tables[i].append(idx)
        
        return tables

    
    
    def fitness(self):
        total_happiness = 0
        for i, row in enumerate(self.repr):
            table_happiness = 0
            combs = list(combinations(row, 2))
            pair_scores = [self.relations_matrix[a, b] for a, b in combs]
            table_happiness = sum(s for s in pair_scores)
            total_happiness += table_happiness
        return total_happiness


In [38]:
solution = WSSolution()

In [39]:
solution.fitness()

np.float64(12800.0)

In [40]:
solution.repr

[[8, 50, 32, 55, 62, 15, 27, 16],
 [0, 34, 10, 30, 44, 4, 26, 40],
 [33, 54, 14, 52, 35, 46, 21, 25],
 [59, 9, 53, 38, 56, 57, 5, 23],
 [61, 2, 29, 24, 63, 11, 42, 19],
 [45, 28, 58, 48, 6, 20, 51, 39],
 [47, 7, 12, 13, 60, 18, 17, 31],
 [22, 43, 36, 49, 3, 1, 37, 41]]

In [45]:
def genetic_algorithm(
    initial_population,
    max_gen,
    selection_algorithm,
    crossover_operator,
    mutation_operator,
    xo_prob=0.9,
    mut_prob=0.1,
    elitism=True,
    maximization=True,
    verbose=False,
):
    population = initial_population
    best_individual = None

    for gen in range(max_gen):
        if verbose:
            print(f"\nGeneration {gen + 1}")

        # Evaluate fitness
        fitnesses = [ind.fitness() for ind in population]

        # Get elite
        if elitism:
            elite = max(population, key=lambda ind: ind.fitness()) if maximization else min(population, key=lambda ind: ind.fitness())

        # Generate new population
        new_population = []

        while len(new_population) < len(population):
            # Selection
            parent1 = selection_algorithm(population)
            parent2 = selection_algorithm(population)

            # Crossover
            if random.random() < xo_prob:
                offspring1, offspring2 = crossover_operator(parent1, parent2)
            else:
                offspring1, offspring2 = deepcopy(parent1), deepcopy(parent2)

            # Mutation
            mutation_operator(offspring1, mut_prob)
            mutation_operator(offspring2, mut_prob)

            new_population.extend([offspring1, offspring2])

        # Replace worst with elite
        if elitism:
            new_population[random.randint(0, len(new_population) - 1)] = elite

        population = new_population[:len(population)]

        # Store best
        candidate = max(population, key=lambda ind: ind.fitness()) if maximization else min(population, key=lambda ind: ind.fitness())
        if best_individual is None or (
            (maximization and candidate.fitness() > best_individual.fitness()) or
            (not maximization and candidate.fitness() < best_individual.fitness())
        ):
            best_individual = deepcopy(candidate)

        if verbose:
            print(f"Best fitness so far: {best_individual.fitness()}")

    return best_individual

def tournament_selection(population, k=3):
    competitors = random.sample(population, k)
    return max(competitors, key=lambda ind: ind.fitness())

def mutate(solution, mut_prob):
    if random.random() < mut_prob:
        tables = solution.repr
        if len(tables) < 2:
            return

        # Pick two random tables
        t1, t2 = random.sample(range(len(tables)), 2)
        if not tables[t1] or not tables[t2]:
            return

        # Swap one person from each
        i1 = random.randint(0, len(tables[t1]) - 1)
        i2 = random.randint(0, len(tables[t2]) - 1)
        tables[t1][i1], tables[t2][i2] = tables[t2][i2], tables[t1][i1]

def crossover(parent1, parent2):
    p1_tables = deepcopy(parent1.repr)
    p2_tables = deepcopy(parent2.repr)
    child1_repr = p1_tables[:len(p1_tables)//2] + p2_tables[len(p2_tables)//2:]
    child2_repr = p2_tables[:len(p2_tables)//2] + p1_tables[len(p1_tables)//2:]

    # Flatten, remove duplicates, fill missing for child1
    def fix_child(tables, num_people, num_tables, cap):
        flat = [p for table in tables for p in table]
        seen = set()
        cleaned = [p for p in flat if not (p in seen or seen.add(p))]
        missing = [i for i in range(num_people) if i not in cleaned]
        cleaned.extend(missing)
        return [cleaned[i*cap:(i+1)*cap] for i in range(num_tables)]

    num_people = len(parent1.relations_matrix)
    cap = parent1.table_capacity
    num_tables = parent1.nr_of_tables

    child1 = WSSolution(relations_matrix=parent1.relations_matrix)
    child2 = WSSolution(relations_matrix=parent1.relations_matrix)
    child1.repr = fix_child(child1_repr, num_people, num_tables, cap)
    child2.repr = fix_child(child2_repr, num_people, num_tables, cap)
    return child1, child2




In [47]:
# Create initial population
population = [WSSolution(relations_matrix=relations_matrix) for _ in range(50)]
for sol in population:
    sol.repr = sol.random_initial_representation()

# Run GA
best = genetic_algorithm(
    initial_population=population,
    max_gen=2000,
    selection_algorithm=tournament_selection,
    crossover_operator=crossover,
    mutation_operator=mutate,
    xo_prob=0.8,
    mut_prob=0.2,
    elitism=True,
    maximization=True,
    verbose=True,
)

print("Best fitness found:", best.fitness())



Generation 1
Best fitness so far: 25800.0

Generation 2
Best fitness so far: 25800.0

Generation 3
Best fitness so far: 25800.0

Generation 4
Best fitness so far: 28700.0

Generation 5
Best fitness so far: 32100.0

Generation 6
Best fitness so far: 32100.0

Generation 7
Best fitness so far: 33000.0

Generation 8
Best fitness so far: 33000.0

Generation 9
Best fitness so far: 33000.0

Generation 10
Best fitness so far: 34200.0

Generation 11
Best fitness so far: 34200.0

Generation 12
Best fitness so far: 34200.0

Generation 13
Best fitness so far: 35000.0

Generation 14
Best fitness so far: 35000.0

Generation 15
Best fitness so far: 36200.0

Generation 16
Best fitness so far: 36200.0

Generation 17
Best fitness so far: 36700.0

Generation 18
Best fitness so far: 36700.0

Generation 19
Best fitness so far: 36800.0

Generation 20
Best fitness so far: 37100.0

Generation 21
Best fitness so far: 38700.0

Generation 22
Best fitness so far: 38700.0

Generation 23
Best fitness so far: 38800

In [503]:
print("Best fitness found:", best.fitness())
print("\nTables:")

for i, table in enumerate(best.repr):
    print(f"Mesa {i+1}: {table}")

Best fitness found: 78700.0

Tables:
Mesa 1: [36, 0, 53, 43, 59, 37, 49, 1]
Mesa 2: [51, 29, 28, 50, 24, 22, 23, 52]
Mesa 3: [6, 25, 26, 56, 8, 7, 27, 57]
Mesa 4: [33, 32, 48, 30, 21, 31, 34, 35]
Mesa 5: [11, 9, 2, 10, 12, 13, 14, 3]
Mesa 6: [39, 40, 42, 54, 55, 41, 44, 38]
Mesa 7: [17, 19, 20, 5, 4, 18, 16, 15]
Mesa 8: [46, 61, 62, 63, 47, 60, 58, 45]


In [389]:
for i in range(10):
    res = 0
    for j in range(10000):
        solution = WSSolution()
        res+=solution.fitness()
    res = res/10000
    print(f"Result {i+1}: Average Fitness: {res}")

Result 1: Average Fitness: 12228.75
Result 2: Average Fitness: 12162.76
Result 3: Average Fitness: 12224.06
Result 4: Average Fitness: 12123.46
Result 5: Average Fitness: 12217.69
Result 6: Average Fitness: 12213.36
Result 7: Average Fitness: 12225.02
Result 8: Average Fitness: 12138.95
Result 9: Average Fitness: 12215.36
Result 10: Average Fitness: 12264.6


In [392]:
for i in range(3):
    res = 0
    scores = []
    for j in range(1000000):
        solution = WSSolution()
        res=solution.fitness()
        scores.append(res)
    avg_score = sum(scores)/1000000
    best_score = max(scores)
    print(f"Result {i+1}: Average Fitness {avg_score} and Best Fitness Found {best_score}")

Result 1: Average Fitness 12206.4644 and Best Fitness Found 35700.0
Result 2: Average Fitness 12202.3834 and Best Fitness Found 33700.0
Result 3: Average Fitness 12207.7375 and Best Fitness Found 33900.0
